## Generating synthetic LSFM data
In this notebook, we generate synthetic LSFM image data and describe the modelling steps of LSFM simulation. 

In [1]:
# Prerequisites: packages & custom functions
from spimagine import volshow
import random
from math import cos, sin, pi, exp, tan
import numpy as np
import biobeam
from scipy import ndimage
import matplotlib.pyplot as plt
from biobeam import Bpm3d
from gputools import fft_convolve, perlin3
from tifffile import imsave, imread
from IPython.display import clear_output
import sys
import os

# Generates a random point on a sphere with radius r
# Input: 
# r - radius of sphere
def randomSpherePoint(r = 1):
    phi = 2*pi*random.random()
    theta = pi*random.random()
    return np.array([cos(phi)*sin(theta),sin(phi)*sin(theta),cos(theta)])*r*random.random()
# Usage: Random displacement of nuclei positions

# Plots a volume using volshow (single call, no additional requirements)
# Input:
# img - 3d image
def plotvol(img):
    %gui qt5
    w = volshow(img) 
    w.set_colormap("hot")
    return w

# Generates the 2D-cross-section of an (angled) cylindrical light sheet.
# Edges are damped accordingly to reduce edge effects.
# Input:
# size   - size of cross-section (2-tuple)
# units  - resolution (μm per pixel) (3-tuple)
# lam    - wavelength of light (in μm)
# theta  - angle of light-sheet from forward direction (in °)
# z_foc  - position of focal plane in forward direction
# NA     - numerical aperture
def angledLightsheet(size = (512,256),units = (.2,)*3, lam = 0.5, theta = 0, z_foc = 0, NA = .1):
     # Transforming angle to radial
    theta = 2*pi*theta/360
    
    [dx,dy,dz] = units
    
    # Calculating wave number in the x-direction
    k0 = 2*pi/lam
    kx = k0*sin(theta)
    
    # Calculating wave number differences
    dkx = 2*pi/(units[0]*size[0])
    dky = 2*pi/(units[1]*size[1])
    
    x,y = np.meshgrid(np.linspace(-size[0]/2,size[0]/2-1,size[0]),np.linspace(-size[1]/2,size[1]/2-1,size[1]))
    
    # Calculating initial field and adjusting propagation direction to be tilted with angle theta
    E_z0 = biobeam.focus_field_cylindrical_plane(shape=(size[0],size[1]),units =  units[0:2],NA = NA, lam = 0.5,z = 0)
    mask = np.multiply(1*(abs(x) > cos(theta)*size[0]/3),np.exp(-0.01*(abs(x)-cos(theta)*size[0]/3))) + 1*(abs(x) <= cos(theta)*size[0]/3)
    E_z0 = np.multiply( E_z0, mask)
    E_z0 = np.multiply( E_z0, np.exp(-1j*kx*dx*x))
    
    # Propagator
    H_dz = -(1j)*np.multiply(np.lib.scimath.sqrt(k0**2-(x*dkx)**2-(y*dky)**2),(dz*z_foc))
    H_dz = np.exp(np.multiply(H_dz,-np.sign(H_dz.real)+1*(H_dz.real == 0)))
    return np.fft.ifft2(np.fft.ifftshift(np.multiply(np.fft.fftshift(np.fft.fft2(E_z0)),H_dz)))

# Generates *reflective index distribution (rid)* and the *fluorescence distribution (fld)*
#for randomly sampled non-overlapping spheres
# Sphere radii are sampled iid from a beta distribution.
# Input:
# size - size of volume (3-tuple)
# n    - targeted number of spheres
# r    - [min. radius, max. radius, beta parameter, glow ring, membrane thickness, separator thikness, nulceus radius]
# p    - probabilities for [scattering, membrane, nucleus]
def SphereGeometry(size = (512,512,512), n = 200, r = [20,40,5,2,3,2,2], p = [0.15,0.2,0.2],distribution='beta'):
    # Initialization
    obj = 0  # number of placed objects
    step = 0 # number of steps
    c = np.zeros((n,3)) # storage for center of spheres/objects
    rc = np.zeros(n)    # storage for radia of spheres
    rid = np.zeros(size,dtype = int) # refractive index distribution (initialization)
    fld = np.zeros(size,dtype = int) # fluorescence distribution (initialization)

    # Here the main function begins:
    while (obj < n) and step <= 3*n:
        step += 1
        # Add a random center
        c[obj,:] = np.array([random.randint(-int(r[0]/2),size[0]+int(r[0]/2)),random.randint(-int(r[0]/2),size[1]+int(r[0]/2)),random.randint(-int(r[0]/2),size[2]+int(r[0]/2))]);
        # Create a radius if none is chosen yet
        if rc[obj] == 0:
            if distribution == 'beta':
                rc[obj] = r[0] + (r[1]-r[0])*np.random.beta(r[2],r[2])
            elif distribution == 'uniform':
                rc[obj] = r[0] + (r[1]-r[0])*np.random.rand()
            else:
                raise ValueError('distribution must be beta or uniform.')
        # Check if center and radius are valid (i.e. no overlap)
        isvalid = True 
        for i in range(obj):
            if np.linalg.norm(c[obj,:]-c[i,:])<rc[obj]+rc[i]:
                isvalid = False
        # Proceed if valid:
        if isvalid:
            isscatter = random.random() < pscatter # is the cell a scatterer?
            hasmembrane = random.random() < pmembrane # has the cell a membrane?
            hasnucleus = random.random() < pnucleus # has the cell a nucleus?
            cnucleus = randomSpherePoint(rc[obj]-rn-rm) # center of the nucleus
            
            # Iterate over bounding box (naive)
            for h in range(max(int(c[obj,0]-rc[obj]-r[3]),0),min(int(c[obj,0]+rc[obj]+r[3]),size[0])):
                for j in range(max(int(c[obj,1]-rc[obj]-r[3]),0),min(int(c[obj,1]+rc[obj]+r[3]),size[1])):
                    for l in range(max(int(c[obj,2]-rc[obj]-r[3]),0),min(int(c[obj,2]+rc[obj]+r[3]),size[2])):
                        dist = np.linalg.norm(c[obj,:]-np.array([h,j,l]))
                        # Glowing ring around the sphere ("glow")
                        if (dist <= rc[obj]+r[3]) and (dist > rc[obj]):
                            fld[h,j,l] = 1 # glow only affect the fluorescence distribution
                        # Membrane of the sphere ("membrane")
                        elif (dist <= rc[obj]) and (dist>rc[obj]-r[4]):
                            fld[h,j,l] = hasmembrane*2 + (1-hasmembrane)*3
                            rid[h,j,l] = hasmembrane*isscatter*2 + (1-hasmembrane)*isscatter*3
                        # Separating "ring" between membrane & cell
                        elif (dist <= rc[obj]) and (dist>rc[obj]-r[4]-r[5]):
                            fld[h,j,l] = 0
                            rid[h,j,l] = 0
                        # Nucleus of the sphere ("nucleus")   
                        elif (np.linalg.norm(c[obj,:]+cnucleus-np.array([h,j,l])) <= r[6]):
                            fld[h,j,l] = hasnucleus*4 + (1-hasnucleus)*3
                            rid[h,j,l] = hasnucleus*isscatter*4 + (1-hasnucleus)*isscatter*3
                        # General sphere / everything else ("general")
                        elif (dist < rc[obj]):
                            fld[h,j,l] = 3
                            rid[h,j,l] = isscatter*3 
            # increase the number of placed spheres/objects                 
            obj += 1 
            clear_output(wait=True)
            print('Current number of spheres: ',str(obj),end='\r')
    # If the number of steps exceed a certain threshold while not being done the algorithm stops.        
    if step == 3*n:
        print('The number of spheres could not be achieved!\n Only ' + str(obj) + ' spheres have been placed.')
    return fld, rid

# Pads an image by zeros
# Input:
# img    - 3d image
# front  - pad size front
# back   - pad size back
# top    - pad size top
# bottom - pad size bottom
# left   - pad size left
# right  - pad size right
def zeroPad3D(img,front,back,top,bottom,left,right):
    nz,nx,ny = img.shape
    res = np.zeros(img.shape+np.array([front+back,top+bottom,left+right,]))
    res[front:front+nz,top:top+nx,left:left+ny] = img
    return res

# "Single-slice" convolution
# x      - 3d image
# f      - 3d filter
# zslice - slice for convolution
def pseudoconvolve(x,f,zslice):
    nz,nx,ny = x.shape
    fz,fx,fy = f.shape
    res = np.zeros((nx,ny))
    #u = zeroPad3D(x,fz//2-1,fz//2,fx//2-1,fx//2,fy//2-1,fy//2)
    zmin = max(0,zslice-fz//2+1)
    zmax = min(nz,zslice+fz//2+1)
    for i in range(nx):
        for j in range(ny):
            u = np.zeros((fz,fx,fy))
            u[max(0,fz//2-zslice-1):min(fz,nz+fz//2-zslice-1),max(0,fx//2-i-1):min(fx,nx+fx//2-i-1),max(0,fy//2-j-1):min(fy,ny+fy//2-j-1)] = \
                    x[zmin:zmax,max(0,i-fx//2+1):min(nx,i+fx//2+1),max(0,j-fy//2+1):min(ny,j+fy//2+1)]
            res[i,j] = np.sum(np.multiply(u,f))
    return res

C:\Users\nikla\anaconda3\envs\BioBeam\lib\site-packages\pyopencl\__init__.py:269: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


## Geometry
1. We create an integer mask. Each integer represents a different part of the geometry such as background, membrane, nucleus, inside and glow.
2. We generate Perlin noise as structure for fore- and background.
3. Calculation of fld and rid.

In [2]:
# 1. Create the masks for the fluorescence and reflective index distributions:
# Select a random seed:
random.seed(123)

# Size parameters
Nx = 256; # size in x-direction (stack-depth)
Ny = 512; # size in y-direction
Nz = 512; # size in z-direction
size = (Nx,Ny,Nz) 

# Parameters for Objects
n = 1519  # number of objects (previously 500)
rmin = 8  # minimum sphere radius (20)
rmax = 15 # maximum sphere radius (40)
rpar = 5  # Parameter for exponential beta distribution: 0.5*(rmax+rmin) + (rmax-rmin) * x ~ beta(rpar,rpar) (5)
dr = 1    # additional fluorescence radius (2)
rm = 1    # membrane width (3)
rs = 1    # separator width (1)
rn = 3    # radius of nucleus (5)
distribution = 'uniform' # distribution to draw radii from [rmin,rmax]
# This yields:
r = [rmin,rmax,rpar,dr,rm,rs,rn]

# Probabilities:
pscatter = 1 # probability of being a scatterer
pmembrane = 1 # probability of having a membrane
pnucleus = 0 # probability of having a nucleus
# Therefore:
p = [pscatter,pmembrane,pnucleus]

# Create masks:
fldmask, ridmask = SphereGeometry(size = size, n = n, r = r, p = p)
imsave(os.getcwd() + '/fldmask.tif',fldmask)
imsave(os.getcwd() + '/ridmask.tif',ridmask)

In [3]:
# 2. Generate Perlin noise
# Select random seed
random.seed(234)

# General parameters
cback = 0.0 # constant background fluorescence
bnoise = 0.05 # intensity of background noise
bstructure = 0.5 # intensity of object structure (noise)

# Perlin scale parameters
bscale = 8. 
sscale = 3.

# Sampling
backnoise = bnoise*perlin3(size = size[::-1], scale = (bscale,bscale,bscale))
structure = bstructure*perlin3(size = size[::-1], scale = (sscale,sscale,sscale))
imsave(os.getcwd() + '/PerlinBackground.tif',backnoise)
imsave(os.getcwd() + '/PerlinStructure.tif',structure)

C:\Users\nikla\anaconda3\envs\BioBeam\lib\site-packages\pyopencl\__init__.py:269: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [4]:
# 3. Calculate fld and rid using the masks
# fluorescing factors
glow = 0.25     # glow around spheres
general = 1     # default 1
membrane = 1.6  # default 1.6
nucleus = 2     # default 2 

# refractive indices
generalrid = complex(0.025,0.001) # default complex(0.025,0.001)
membranerid = complex(0.03,0.0015) # default complex(0.03,0.0015) 
nucleusrid = complex(0.035,0.002) # default complex(0.035,0.002)

# Generation of fld and rid
fld = np.multiply(1*(fldmask <= 1),backnoise+cback) + glow*(fldmask == 1) + np.multiply(1*(fldmask == 2),structure+membrane) \
        + np.multiply(1*(fldmask == 4),structure+nucleus) + np.multiply(1*(fldmask == 3),structure+general)

rid = np.multiply(1*(ridmask == 2),membranerid) + np.multiply(1*(ridmask == 4),nucleusrid) + np.multiply(1*(ridmask == 3),generalrid)

# Light Propagation
Calculate the light transport and mimic the LSFM imaging process. We offer the possibility to perform "multidirectional" illumination by specifying a number of directions for illumination.

In [5]:
%%capture

# directions in deg (array)
angle = [0] # default [0], bi-directional e.g. [-5,5]
            # multidirectional e.g. [-5,-4,-3,-2,-1,0,1,2,3,4,5]

# Light wavelength
lmbd = 0.5 # default: 0.5

# Aperture sizes for detection and the cylindrical light sheets
NAdet = 0.5 # default: 0.95
NAls = 0.1 # default: 0.1

# Sampling in z-direction 
zsample = 1
# Focal plane distance
z_foc = .5*size[2]

# (efficiency) Set a maximum kernel size/height for the detection PSF:
kerint = 64 

# Create geometry 
# Space of each pixel in mikrons (mikrometer)
units = (.4,)*3 # (.2)
m = Bpm3d(dn = rid.transpose(2,0,1), units = units, lam = lmbd) 
m_ref = Bpm3d(dn = np.zeros(size).transpose(2,0,1), units = units, lam = lmbd)  

# Create detection beam
v = np.zeros([min(size[0],kerint),size[1],size[2]])
v[:,size[1]//2-min(size[1]//2,kerint//2):size[1]//2+min(size[1]//2,kerint//2),
  size[2]//2-min(size[2]//2,kerint//2):size[2]//2+min(size[2]//2,kerint//2)] = biobeam.focus_field_beam(shape = [min(size[2],kerint),min(size[1],kerint),min(size[0],kerint)],
                                                                                                  units = units,NA = NAdet, n0 = 1.33)
# Illumination results (initialization)
res = np.zeros(((size[0]-kerint)//zsample,size[1],size[2]))
ref = np.zeros(((size[0]-kerint)//zsample,size[1],size[2]))

# Light propagation simulation:
for k in range((size[0]-kerint)//zsample): # iterate over all imaged slices
    # illumination response
    u = np.zeros([size[2],size[0],size[1]])
    u_ref = np.zeros([size[2],size[0],size[1]])
    for l in range(len(angle)): 
        E_z = angledLightsheet(size = (size[1],size[0]),units = units, lam = lmbd, theta = 0, z_foc = z_foc, NA = NAls)
        u += m.propagate(u0 = np.roll(E_z/(2*angle[l]+1),zsample*k-size[0]//2+kerint//2,0), return_comp = "intens")
        u_ref += m_ref.propagate(u0 = np.roll(E_z/(2*angle[l]+1),zsample*k-size[0]//2+kerint//2,0), return_comp = "intens")
    u = np.multiply(u[:,zsample*k:zsample*k+kerint,:],fld[zsample*k:zsample*k+kerint,:,:].transpose(2,0,1)).transpose(1,2,0)
    u = fft_convolve(u,v)
    u_ref = np.multiply(u_ref[:,zsample*k:zsample*k+kerint,:],fld[zsample*k:zsample*k+kerint,:,:].transpose(2,0,1)).transpose(1,2,0)
    u_ref = fft_convolve(u_ref,v)
    res[k,:,:] = u[kerint//2,:,:]
    ref[k,:,:] = u_ref[kerint//2,:,:]

# Currently -> fast but expensive in memory
# Alternatively comment out the code lines for reference including m_ref and use
# the following lines of code (slower but lower memory intensive)
# m = Bpm3d(dn = np.zeros(size).transpose(2,0,1), units = units, lam = lmbd)  
# for k in range((size[0]-kerint)//zsample): 
#    u = np.zeros([size[2],size[0],size[1]])
#    for l in range(len(angle)): 
#        E_z = angledLightsheet(size = (size[1],size[0]),units = units, lam = lmbd, theta = 0, z_foc = z_foc, NA = NAls)
#        u += m.propagate(u0 = np.roll(E_z/(2*angle[l]+1),zsample*k-size[0]//2+kerint//2,0), return_comp = "intens")
#    u = np.multiply(u[:,zsample*k:zsample*k+kerint,:],fld[zsample*k:zsample*k+kerint,:,:].transpose(2,0,1)).transpose(1,2,0)
#    u = fft_convolve(u,v)
#    ref[k,:,:] = u_ref[kerint//2,:,:]

imsave(os.getcwd() + '/Result.tif',res)
imsave(os.getcwd() + '/Reference.tif',ref)

In [6]:
# Display result or reference
plotvol(res)